# MRMr explained
This notebook seeks to explain and provide an example of the Min-redundancy max-relevance algorithms proposed by Chris Ding and Hanchuan Peng in their paper *Minimum Redundancy Feature Selection from Microarray
Gene Expression Data* from 2005. 

This paper might be over 15 years old, but the ideas are still relevant. Feature selection is as important as ever, as well as being able to answer the question: which features in my dataset are the most important?
We live in a golden age of computing power. 
By selecting an effective subset of features, you:
- Reduce training time, as the size of the feature vectors are smaller. 
- Create more interpretable models. Simpler models are easier to interpret.
- Improve accuracy, especially if the original featureset contains features that are no better than noise. 
- Reduce the risk of overfitting. Simpler models with smaller featuresets are less likely to overfit to the training data.

## What is MRMr?
MRMr is an algorithm that ranks features based on importance to predicting the target. It is an iterative algorithm, selecting one feature at a time. It calculates a value based on either a difference or quotient basis of the relevance and the redundancy of the feature.
- Relevance: how well does the feature correlate to the target? 
    - For categorical variables, this is based on mutual information. 
    - For continuous variables it is the F-statistic of that variable.
- Redundancy: how well does the feature correlate to features selected in previous iterations? The idea is that if a candidate feature correlates correlates strongly to an already selected feature, then it won't provide much additional information to the feature set.

The first feature chosen is the one that is most relevant, with no redundancy constraints: there are no other features to compare and see if the feature is redundant.


## Caveats
- MRMr only works for classification.
- All features must be either entirely categorical or numerical. It is still usable if you have a mix, but some preprocessing might be required, such as binning the numerical values.



## Example

As an example, I loaded the wine dataset from sklearn. This dataset contains measurements for three different types of wines grown in the same region in Italy. It contains 13 numeric features.

In [81]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_wine
from scipy.stats import f_oneway
from numpy import corrcoef
import operator
from sklearn.ensemble import RandomForestClassifier

In [48]:
data = load_wine(as_frame=True)

In [63]:
df = pd.concat([data.data, data.target], axis=1)

In [64]:
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


In [136]:
class MRMR:
    def __init__(self, df, target_name, difference_or_quotient='difference'):
        self.df = df
        self.idxs_by_class = [df[df[target_name]==v].index for v in df[target_name].unique()]
        self.features = [col for col in df.columns if col!=target_name]
        self.ranked_features = []
        self.feature_relevance = {feat_name: self.calc_feature_relevance(self.df[feat_name]) for feat_name in self.features}
        self.use_difference = difference_or_quotient=='difference'
        self.calculated_correlations = {}
        
    def calc_feature_relevance(self, feature):
        groups = [feature[class_idxs].values for class_idxs in self.idxs_by_class]
        return f_oneway(*groups).statistic
    
    def calc_feature_redundance(self, feature):
        redundance = 0
        for feat in self.ranked_features:
            if (feat, feature) not in self.calculated_correlations:
                self.calculated_correlations[(feat, feature)] = abs(corrcoef(self.df[feature], self.df[feat])[1,0])
                self.calculated_correlations[(feature, feat)] = abs(corrcoef(self.df[feature], self.df[feat])[1,0])
            
            redundance += self.calculated_correlations[(feat, feature)]
        return redundance   
    
    def rank_features(self):
        most_important_feature = max(self.feature_relevance.items(), key=operator.itemgetter(1))[0]
        self.ranked_features.append(most_important_feature)
        
        while len(self.ranked_features) != len(self.features):
            top_importance = 0
            most_important_feature = None
            for feat in self.features:
                if feat in self.ranked_features:
                    continue
                    
                feature_redundancy = self.calc_feature_redundance(feat)
                feature_relevance = self.feature_relevance[feat]
                if self.use_difference:
                    importance = feature_relevance - feature_redundancy
                else:
                    importance = feature_relevance / feature_redundancy
                
                if importance > top_importance:
                    top_importance = importance
                    most_important_feature = feat
                    
            self.ranked_features.append(most_important_feature)
            
        return self.ranked_features
                
                

In [137]:
mrmr = MRMR(df, 'target')
mrmr.rank_features()

['flavanoids',
 'proline',
 'od280/od315_of_diluted_wines',
 'alcohol',
 'color_intensity',
 'hue',
 'total_phenols',
 'malic_acid',
 'alcalinity_of_ash',
 'proanthocyanins',
 'nonflavanoid_phenols',
 'ash',
 'magnesium']

In [138]:
mrmr = MRMR(df, 'target', difference_or_quotient='quotient')
mrmr.rank_features()

['flavanoids',
 'color_intensity',
 'proline',
 'od280/od315_of_diluted_wines',
 'alcohol',
 'hue',
 'total_phenols',
 'alcalinity_of_ash',
 'malic_acid',
 'proanthocyanins',
 'nonflavanoid_phenols',
 'ash',
 'magnesium']

In [139]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000)
model.fit(df[mrmr.features], df['target'])
pd.Series(model.feature_importances_, index=mrmr.features).sort_values(ascending=False)

proline                         0.167721
color_intensity                 0.167566
flavanoids                      0.154224
od280/od315_of_diluted_wines    0.117799
alcohol                         0.116276
hue                             0.082281
total_phenols                   0.055714
magnesium                       0.033280
malic_acid                      0.031723
alcalinity_of_ash               0.028569
proanthocyanins                 0.021047
ash                             0.013025
nonflavanoid_phenols            0.010776
dtype: float64